# Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.metrics import mean_absolute_error
import tensorflow as tf
import optuna
import os
import random
import datetime

np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)

In [2]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# Data

In [3]:
data_path = 'https://raw.githubusercontent.com/antbartash/max_temp/master/data/data2.csv'
data = pd.read_csv(data_path, index_col=0)
data['DATE'] = data['DATE'].astype('datetime64[ns]')

print(data.shape)
data.head()

(40898, 4)


,STATION,NAME,DATE,TMAX
0,USW00012916,"NEW ORLEANS AIRPORT, LA US",2010-01-01,12.2
1,USW00012916,"NEW ORLEANS AIRPORT, LA US",2010-01-02,10.6
2,USW00012916,"NEW ORLEANS AIRPORT, LA US",2010-01-03,8.3
3,USW00012916,"NEW ORLEANS AIRPORT, LA US",2010-01-04,6.1
4,USW00012916,"NEW ORLEANS AIRPORT, LA US",2010-01-05,6.1


In [4]:
data = data[['STATION', 'DATE', 'TMAX']]

# add sine and cosine transforms to add periodicality
doy = data['DATE'].dt.dayofyear / 365.25
data['Year_sin'] = np.sin(doy * 2 * np.pi)
data['Year_cos'] = np.cos(doy * 2 * np.pi)

print(data.shape)
data.head()

(40898, 5)


,STATION,DATE,TMAX,Year_sin,Year_cos
0,USW00012916,2010-01-01,12.2,0.017202,0.999852
1,USW00012916,2010-01-02,10.6,0.034398,0.999408
2,USW00012916,2010-01-03,8.3,0.051584,0.998669
3,USW00012916,2010-01-04,6.1,0.068755,0.997634
4,USW00012916,2010-01-05,6.1,0.085906,0.996303


## Data preprocessing

In [5]:
# TRAIN/VALID/TEST SPLIT

train_df = data.loc[data['DATE'].dt.year <= 2021].reset_index(drop=True).copy()
valid_df = data.loc[data['DATE'].dt.year == 2022].reset_index(drop=True).copy()
test_df = data.loc[data['DATE'].dt.year == 2023].reset_index(drop=True).copy()

print(f'Train: {train_df.shape}')
print(f'Valid: {valid_df.shape}')
print(f'Test: {test_df.shape}')

Train: (35058, 5)
Valid: (2920, 5)
Test: (2920, 5)


In [6]:
# SCALING

train_df.drop(columns=['DATE'], inplace=True)
valid_df.drop(columns=['DATE'], inplace=True)
test_df.drop(columns=['DATE'], inplace=True)

# keep station to drop mixed windows later (encode to avoid errors from scaler)
encoder = OrdinalEncoder()
encoder.fit(train_df[['STATION']])
train_df['STATION'] = encoder.transform(train_df[['STATION']])[:, 0]
valid_df['STATION'] = encoder.transform(valid_df[['STATION']])[:, 0]
test_df['STATION'] = encoder.transform(test_df[['STATION']])[:, 0]

# scaling
scaler = StandardScaler()
scaler.fit(train_df)
train_df = pd.DataFrame(scaler.transform(train_df),
                        columns=scaler.feature_names_in_, index=train_df.index)
valid_df = pd.DataFrame(scaler.transform(valid_df),
                        columns=scaler.feature_names_in_, index=valid_df.index)
test_df = pd.DataFrame(scaler.transform(test_df),
                       columns=scaler.feature_names_in_, index=test_df.index)

train_df.head()

,STATION,TMAX,Year_sin,Year_cos
0,-1.091014,-0.747812,0.024401,1.414059
1,-1.091014,-0.896450,0.048721,1.413431
2,-1.091014,-1.110117,0.073027,1.412386
3,-1.091014,-1.314495,0.097312,1.410922
4,-1.091014,-1.314495,0.121567,1.409041


In [7]:
def tmax_inverse_transform(value, scale=scaler.scale_[1], mean=scaler.mean_[1]):
    return value * scale + mean

In [8]:
# TF DATASET

def create_dataset(data, target_col, source_col, seq_length):
  input_data = data[:-seq_length]
  # adding source_col to target allows to drop samples with features and target
  # from diff sources (source_col will be dropped from the target later)
  targets = data[[source_col, target_col]][seq_length:]
  dataset = tf.keras.preprocessing.timeseries_dataset_from_array(
      input_data, targets,
      sequence_length=seq_length,
      sequence_stride=1,
      batch_size=32,
      shuffle=False,
      seed=42
    )
  for batch in dataset:
    inputs, targets = batch
    assert np.array_equal(inputs[0], data[:seq_length])  # First sequence: steps [0-13]
    # Corresponding target: step 14
    assert np.array_equal(targets[0, 1], data.loc[seq_length, target_col])
    break
  return dataset

train_ds = create_dataset(train_df, 'TMAX', 'STATION', 14)
valid_ds = create_dataset(valid_df, 'TMAX', 'STATION', 14)
test_ds = create_dataset(test_df, 'TMAX', 'STATION', 14)

In [9]:
# DATASET CLEANING

def filter_mixed_windows(dataset):
    def is_valid_window(inputs, targets):
        # Extract the first feature (station) from inputs and targets
        input_station_ids = inputs[:, :, 0]  # Shape: (batch_size, sequence_length)
        target_station_ids = targets[:, 0]  # Shape: (batch_size)

        # Check if all station IDs in the inputs are the same
        input_same_station = tf.reduce_all(tf.reduce_max(input_station_ids, axis=1) == tf.reduce_min(input_station_ids, axis=1))

        # Check if the target's station ID matches the input station ID
        target_matches_input = tf.reduce_all(tf.reduce_max(input_station_ids, axis=1) == target_station_ids)

        # Only keep windows where both conditions are true
        return tf.logical_and(input_same_station, target_matches_input)

    # Filter the dataset
    filtered_dataset = dataset.filter(is_valid_window)
    return filtered_dataset

train_ds = filter_mixed_windows(train_ds)
valid_ds = filter_mixed_windows(valid_ds)
test_ds = filter_mixed_windows(test_ds)

In [10]:
def drop_first_column(feature, label):
    feature = feature[:, :, 1:]  # Keep all rows, drop the first column
    label = label[:, 1:]    # Keep all rows, drop the first column
    return feature, label

train_ds = train_ds.map(drop_first_column)
valid_ds = valid_ds.map(drop_first_column)
test_ds = test_ds.map(drop_first_column)

In [11]:
def load_data(batch_size, prefetch=tf.data.AUTOTUNE,
              train_ds=train_ds, valid_ds=valid_ds, test_ds=test_ds):
    train_ds = train_ds.rebatch(batch_size).prefetch(prefetch)
    valid_ds = valid_ds.rebatch(batch_size).prefetch(prefetch)
    test_ds = test_ds.rebatch(batch_size).prefetch(prefetch)
    train_num_batches = len(list(train_ds))
    valid_num_batches = len(list(valid_ds))
    test_num_batches = len(list(test_ds))
    # print(f'load_data - num_batches (train, valid, test): {train_num_batches}, {valid_num_batches}, {test_num_batches}')
    train_ds_repeat = train_ds.repeat()
    valid_ds_repeat = valid_ds.repeat()
    test_ds_repeat = test_ds.repeat()
    return train_ds_repeat, valid_ds_repeat, test_ds_repeat, train_num_batches, valid_num_batches, test_num_batches

# Model

In [12]:
def create_model(trial, input_shape):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Input(shape=input_shape))
    # Shape: (time, features) => (time*features)
    tf.keras.layers.Reshape(target_shape=(-1,)), # otherwise, the model is applied to each input time step
    
    # layer 0
    n_units_0 = trial.suggest_int('nunints_layer_0', 32, 2048, step=32)
    actv_func_0 = trial.suggest_categorical('actv_func_layer_0', ['relu', 'leaky_relu', 'elu', 'sigmoid', 'tanh', 'gelu'])

    bias_reg_init_0 = trial.suggest_categorical('bias_reg_init_layer_0', ['l1', 'l2', 'l1l2', None])
    if bias_reg_init_0 == 'l1':
        bias_reg_0 = tf.keras.regularizers.l1(trial.suggest_float('bias_reg_layer_0', 0.0001, 0.5))
    elif bias_reg_init_0 == 'l2':
        bias_reg_0 = tf.keras.regularizers.l2(trial.suggest_float('bias_reg_layer_0', 0.0001, 0.5))
    elif bias_reg_init_0 == 'l1l2':
        bias_reg_0 = tf.keras.regularizers.L1L2(trial.suggest_float('bias_reg_layer_0_l1', 0.0001, 0.5),
                                                trial.suggest_float('bias_reg_layer_0_l2', 0.0001, 0.5))
    else:
        bias_reg_0 = None

    kernel_reg_init_0 = trial.suggest_categorical('kernel_reg_init_layer_0', ['l1', 'l2', 'l1l2', None])
    if kernel_reg_init_0 == 'l1':
        kernel_reg_0 = tf.keras.regularizers.l1(trial.suggest_float('kernel_reg_layer_0', 0.0001, 0.5))
    elif kernel_reg_init_0 == 'l2':
        kernel_reg_0 = tf.keras.regularizers.l2(trial.suggest_float('kernel_reg_layer_0', 0.0001, 0.5))
    elif kernel_reg_init_0 == 'l1l2':
        kernel_reg_0 = tf.keras.regularizers.L1L2(trial.suggest_float('kernel_reg_layer_0_l1', 0.0001, 0.5),
                                                  trial.suggest_float('kernel_reg_layer_0_l2', 0.0001, 0.5))
    else:
        kernel_reg_0 = None

    kernel_initializer_0 = trial.suggest_categorical(
        'kernel_initializer_layer_0', ['glorot_uniform', 'glorot_normal',
                                       'he_uniform', 'he_normal',
                                       'lecun_uniform', 'lecun_normal']
    )
    
    model.add(
        tf.keras.layers.Dense(
            units=n_units_0, activation=actv_func_0, 
            kernel_initializer=kernel_initializer_0,
            bias_regularizer=bias_reg_0,
            kernel_regularizer=kernel_reg_0
        )
    )

    
    # hidden layers
    num_layers = trial.suggest_int('num_layers', 0, 3)
    batch_norm = trial.suggest_categorical(f'batch_norm', [True, False])
    for layer_num in range(num_layers):
        layer_i = layer_num + 1
        n_units = trial.suggest_int(f'nunits_layer_{layer_i}', 32, 2048, step=32)
        actv_func = trial.suggest_categorical(f'actv_func_layer_{layer_i}', ['relu', 'leaky_relu', 'elu', 'sigmoid', 'tanh', 'gelu'])

        bias_reg_init = trial.suggest_categorical(f'bias_reg_init_layer_{layer_i}', ['l1', 'l2', 'l1l2', None])
        if bias_reg_init == 'l1':
            bias_reg = tf.keras.regularizers.l1(trial.suggest_float(f'bias_reg_layer_{layer_i}', 0.0001, 0.5))
        elif bias_reg_init == 'l2':
            bias_reg = tf.keras.regularizers.l2(trial.suggest_float(f'bias_reg_layer_{layer_i}', 0.0001, 0.5))
        elif bias_reg_init == 'l1l2':
            bias_reg = tf.keras.regularizers.L1L2(trial.suggest_float(f'bias_reg_layer_{layer_i}_l1', 0.0001, 0.5),
                                                  trial.suggest_float(f'bias_reg_layer_{layer_i}_l2', 0.0001, 0.5))
        else:
            bias_reg = None

        kernel_reg_init = trial.suggest_categorical(f'kernel_reg_init_layer_{layer_i}', ['l1', 'l2', 'l1l2', None])
        if kernel_reg_init == 'l1':
            kernel_reg = tf.keras.regularizers.l1(trial.suggest_float(f'kernel_reg_layer_{layer_i}', 0.0001, 0.5))
        elif kernel_reg_init == 'l2':
            kernel_reg = tf.keras.regularizers.l2(trial.suggest_float(f'kernel_reg_layer_{layer_i}', 0.0001, 0.5))
        elif kernel_reg_init == 'l1l2':
            kernel_reg = tf.keras.regularizers.L1L2(trial.suggest_float(f'kernel_reg_layer_{layer_i}_l1', 0.0001, 0.5),
                                                    trial.suggest_float(f'kernel_reg_layer_{layer_i}_l2', 0.0001, 0.5))
        else:
            kernel_reg = None

        kernel_initializer = trial.suggest_categorical(
            f'kernel_initializer_layer_{layer_i}', ['glorot_uniform', 'glorot_normal',
                                                    'he_uniform', 'he_normal',
                                                    'lecun_uniform', 'lecun_normal']
        )
        
        if batch_norm:
            model.add(tf.keras.layers.BatchNormalization())
        dropout_rate = trial.suggest_float(f'dropout_rate_layer_{layer_i}', 0.0, 0.999)
        model.add(tf.keras.layers.Dropout(dropout_rate))
        model.add(
            tf.keras.layers.Dense(
                n_units, actv_func, 
                kernel_initializer=kernel_initializer,
                bias_regularizer=bias_reg,
                kernel_regularizer=kernel_reg
            )
        )
    if batch_norm:
        model.add(tf.keras.layers.BatchNormalization())
    dropout_rate = trial.suggest_float(f'dropout_rate_layer_output', 0.0, 0.999)
    model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.Dense(1, activation='linear'))
    return model

# Optimizer

In [13]:
def create_optimizer(trial):
    opt_kwargs = {}
    opt_init = trial.suggest_categorical('optimizer', ['SGD', 'Adam', 'Nadam', 'Adamax'])
    if opt_init == 'SGD':
        opt_kwargs['learning_rate'] = trial.suggest_float('opt_lr', 1e-2, 1e-1, log=True)
        opt_kwargs['momentum'] = trial.suggest_float('opt_momentum', 1e-5, 0.1, log=True)
        opt_kwargs['nesterov'] = trial.suggest_categorical('opt_nesterov', [True, False])
    if opt_init == 'Adam':
        opt_kwargs['learning_rate'] = trial.suggest_float('opt_lr', 1e-2, 1e-1, log=True)
        opt_kwargs['beta_1'] = trial.suggest_categorical('opt_beta_1', [0.9, 0.95, 0.99, 0.999])
        opt_kwargs['beta_2'] = trial.suggest_categorical('opt_beta_2', [0.9, 0.95, 0.99, 0.999])
    if opt_init == 'Nadam':
        opt_kwargs['learning_rate'] = trial.suggest_float('opt_lr', 1e-2, 1e-1, log=True)
        opt_kwargs['beta_1'] = trial.suggest_categorical('opt_beta_1', [0.9, 0.95, 0.99, 0.999])
        opt_kwargs['beta_2'] = trial.suggest_categorical('opt_beta_2', [0.9, 0.95, 0.99, 0.999])
    if opt_init == 'Adamax':
        opt_kwargs['learning_rate'] = trial.suggest_float('opt_lr', 1e-2, 1e-1, log=True)
        opt_kwargs['beta_1'] = trial.suggest_categorical('opt_beta_1', [0.9, 0.95, 0.99, 0.999])
        opt_kwargs['beta_2'] = trial.suggest_categorical('opt_beta_2', [0.9, 0.95, 0.99, 0.999])
    optimizer = getattr(tf.optimizers, opt_init)(**opt_kwargs)
    return optimizer

# Objective/train

In [14]:
INPUT_SHAPE = (14, 3, )

def objective(trial):
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256, 512, 1024])
    train_ds_repeat, valid_ds_repeat, _, train_num_batches, valid_num_batches, _ = load_data(batch_size=batch_size)
    
    model = create_model(trial, input_shape=INPUT_SHAPE)
    optimizer = create_optimizer(trial)
    model.compile(
        loss=tf.keras.losses.MeanAbsoluteError(),
        optimizer=optimizer,
        metrics=[tf.keras.metrics.MeanSquaredError()]
    )
    
    # callbacks
    logdir = os.path.join("logs/optuna", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
    tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
    earlystopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=25)
    lr_scheduler_callback = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss', patience=10,
        factor=trial.suggest_float('lr_scheduler_factor', 0.1, 0.75)
    )

    history = model.fit(
        train_ds_repeat, epochs=500, validation_data=valid_ds_repeat,
        steps_per_epoch=train_num_batches, validation_steps=valid_num_batches,
        callbacks=[tensorboard_callback, earlystopping_callback, lr_scheduler_callback],
        verbose=0
    )
    print('\n')
    return np.min(history.history['val_loss'])

# Run

In [ ]:
sampler = optuna.samplers.TPESampler(
    n_startup_trials=25, n_ei_candidates=24,
    multivariate=False, seed=42
)
study = optuna.create_study(direction='minimize', sampler=sampler, study_name='study', storage='sqlite:///db.sqlite3')
study.optimize(
    objective, n_trials=1000,
    timeout=3600*11, # in seconds
    n_jobs=1,
    show_progress_bar=True
)

[I 2024-12-28 14:28:41,650] A new study created in RDB with name: study


  0%|          | 0/1000 [00:00<?, ?it/s]



[I 2024-12-28 14:38:37,855] Trial 2 finished with value: 5.695906639099121 and parameters: {'batch_size': 128, 'nunints_layer_0': 1792, 'actv_func_layer_0': 'tanh', 'bias_reg_init_layer_0': None, 'kernel_reg_init_layer_0': 'l1l2', 'kernel_reg_layer_0_l1': 0.49476554881877133, 'kernel_reg_layer_0_l2': 0.3454267371894703, 'kernel_initializer_layer_0': 'glorot_normal', 'num_layers': 3, 'batch_norm': True, 'nunits_layer_1': 736, 'actv_func_layer_1': 'tanh', 'bias_reg_init_layer_1': 'l1l2', 'bias_reg_layer_1_l1': 0.46613244387080294, 'bias_reg_layer_1_l2': 0.4979830681467701, 'kernel_reg_init_layer_1': 'l1l2', 'kernel_reg_layer_1_l1': 0.0643566692325475, 'kernel_reg_layer_1_l2': 0.32464270915398447, 'kernel_initializer_layer_1': 'glorot_uniform', 'dropout_rate_layer_1': 0.6426288456562775, 'nunits_layer_2': 192, 'actv_func_layer_2': 'gelu', 'bias_reg_init_layer_2': None, 'kernel_reg_init_layer_2': 'l2', 'kernel_reg_layer_2': 0.3582606759893285, 'kernel_initializer_layer_2': 'glorot_normal